**[필요한 라이브러리 호출 및 API키 설정]**

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")    # https://dashboard.cohere.com/ (필요하시면 말씀하세요)

In [2]:
from langchain.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables.history import BaseChatMessageHistory, RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate

c:\Users\user\Desktop\MODULABS\LangchainThon\LangchainThon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## **[데이터 임베딩]**

**[청크 분할]**

In [3]:
# Chunk split
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    separators=["\n\n", "\n", " ", ""]             
)

**[PDF 문서 로드/분할 및 벡터 임베딩]**

In [4]:
from langchain_community.document_loaders import PyPDFLoader

PDF_PATH = r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\pdf\모두연 브랜딩북 정리.pdf"

loader_pdf = PyPDFLoader(PDF_PATH)
pages_pdf = loader_pdf.load()

for d in pages_pdf:
    d.metadata["source_type"] = "pdf"
    d.metadata["source"] = os.path.basename(PDF_PATH)

docs_pdf = text_splitter.split_documents(pages_pdf)

**[HTML 문서 로드/분할 및 벡터 임베딩]**

In [5]:
from langchain_community.document_loaders import UnstructuredHTMLLoader

HTML_PATH = [
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\출결 및 공가에 대하여 23f2d25db6248149a6faf3763fbcfdae.html",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\데싸 5기 훈련 정보 23f2d25db62481f3aa6bc6dbdf2fc11c.html"
]

html_list = []

# 각 파일 로드 + Metadata 저장
for path in HTML_PATH:
    loader_html = UnstructuredHTMLLoader(path)
    pages_html = loader_html.load()

    for d in pages_html:
        d.metadata["source_type"] = "html"
        d.metadata["source"] = os.path.basename(path)

    html_list.extend(pages_html)  

docs_html = text_splitter.split_documents(html_list)

**[WORD 문서 로드/분할 및 벡터 임베딩]**

In [6]:
from langchain_community.document_loaders import Docx2txtLoader

WORD_PATH = r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\word\휴가신청서(데싸_5기).docx"
loader_word = Docx2txtLoader(WORD_PATH)
pages_word = loader_word.load()

for d in pages_word:
    d.metadata["source_type"] = "word"
    d.metadata["source"] = os.path.basename(WORD_PATH)

docs_word = text_splitter.split_documents(pages_word)

**[CSV 문서 로드/분할 및 벡터 임베딩]**

In [7]:
from langchain_community.document_loaders.csv_loader import CSVLoader

CSV_PATH = [
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\csv\데이터 사이언티스트 5기 일정표.csv",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\csv\데싸 5기 동료들.csv",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\csv\데싸 5기 운영진.csv"
]

csv_list = []

# 각 파일 로드
for path in CSV_PATH:
    loader_csv = CSVLoader(path, encoding = 'cp949')
    pages_csv = loader_csv.load()

    for d in pages_csv:
        d.metadata["source_type"] = "csv"
        d.metadata["source"] = os.path.basename(path)

    csv_list.extend(pages_csv)  

docs_csv = text_splitter.split_documents(csv_list)

**[벡터db 저장]**

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.document_transformers import LongContextReorder
from langchain.retrievers.document_compressors import DocumentCompressorPipeline

# ChromaDB 벡터 임베딩 후 저장
vectorstore = Chroma.from_documents(docs_html, OpenAIEmbeddings(model='text-embedding-3-small'))

vectorstore.add_documents(docs_word)
vectorstore.add_documents(docs_csv)
vectorstore.add_documents(docs_pdf)

## **[RAG 시스템]**

**[채팅 히스토리와 사용자 질문 통합]**

In [ ]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.retrievers.document_compressors import LLMChainExtractor
from datetime import datetime, timezone
from zoneinfo import ZoneInfo


# =========== Cohere API KEY 필요 ============

# Reranking 이전 base 
base_retriever = vectorstore.as_retriever(
    search_type="mmr", 
    search_kwargs={"lambda_mult": 0.4, "fetch_k": 96, "k": 48}
)

# Rerank
reranker = CohereRerank(
    model="rerank-multilingual-v3.0",    
    top_n=10                              
)

# Reorder
reorder = LongContextReorder()

# Rerank + Reorder
compressor = DocumentCompressorPipeline(transformers=[reranker, reorder])

upgraded_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor            
)
# ===========================================




# ======== Cohere API KEY 없는 경우 ==========

# retriever = vectorstore.as_retriever(
#     search_type="mmr",                      
#     search_kwargs={"lambda_mult": 0.3, "fetch_k": 32, "k": 8, }  
# )

# # Long-Context Reorder 활용하여 컨텍스트 재정렬
# reordering = LongContextReorder()
# compressor = DocumentCompressorPipeline(transformers=[reordering])

# reordered_retriever = ContextualCompressionRetriever(
#     base_retriever=retriever,
#     base_compressor=compressor
# )
# ===========================================


# LLM 모델 선언
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 현재 시간 선언
KST = ZoneInfo("Asia/Seoul")
today_str = datetime.now(KST).strftime("%Y-%m-%d %H:%M:%S")

# 질문 프롬프트
contextualize_q_system_prompt = """

이전 대화가 있다면 참고하여,
사용자의 최신 질문을 독립적으로 이해 가능한 한 문장으로 바꿔주세요.
답변하지 말고 질문만 재작성하세요.

"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


history_aware_retriever = create_history_aware_retriever(
    llm,
    upgraded_retriever,             # Cohere API KEY 없는 경우 >> reordered_retriever
    contextualize_q_prompt
)


# 답변 프롬프트
qa_system_prompt = """

당신은 '모두의연구소(모두연)' 수강생들의 비서입니다.
현재 시간은 {today} (KST)입니다. 사용자의 '어제, 내일' 등의 표현은 {today}를 기준으로 파악하세요.
제공된 문서 내용만을 근거로 답하세요. 근거가 없으면 '문서에서 관련 내용이 확인되지 않습니다.'라고만 대답하세요.
사용자 입력에 포함된 사실은 근거로 사용하지 마세요.
최대 3문장으로 짧게 답변하세요.

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

**[RAG 체인 구축]**

In [10]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

**[세션별 기록 저장]**

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

#채팅 세션별 기록 저장 위한 Dictionary 선언
store = {}

#주어진 session_id 값에 매칭되는 채팅 히스토리 가져오는 함수 선언
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


#RunnableWithMessageHistory 모듈로 rag_chain에 채팅 기록 세션별로 자동 저장 기능 추가
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

## **[실험 결과]**

### **[RUN 1]**

- 성능: 테스트 6,7,8 정답 / 나머지 데이터 확인 불가

- textsplitter
    - chunk_size=512  
    - chunk_overlap=50  

- retriever: 
    - type: mmr
    - kwargs
        - lambda_mult: 0.5
        - fetch_k: 24
        - k: 6

### **[RUN 2]**

- 성능: 테스트 1, 6, 7, 8 정답 / 2, 3 부분 정답(오답 섞임) / 4, 5 데이터 확인 불가

- textsplitter
    - chunk_size=512  
    - chunk_overlap=50  

- retriever: 
    - type: mmr
    - kwargs
        - lambda_mult: 0.3
        - fetch_k: 32
        - k: 8
    - LongContext Reorder
        - k: 32
        - top_n: 8
        - model: rerank-multilingual-v3.0

### **[RUN 3]**

- 성능: 테스트 1, 4, 6, 7, 8 정답 / 2, 3 부분 정답(오답 섞임) / 5 데이터 확인 불가

- textsplitter
    - chunk_size=512  
    - chunk_overlap=50  

- retriever
    - type: similarity
    - kwargs
        - k: 32
    - cohere rerank
        - top_n: 8
        - model: rerank-multilingual-v3.0

### **[RUN 4]**

- 성능: 테스트 1, 2, 3, 4, 6, 7, 8 정답 / 5 데이터 확인 불가

- textsplitter
    - chunk_size=512  
    - chunk_overlap=50  

- retriever: 
    - type: mmr
    - kwargs
        - lambda_mult: 0.3
        - fetch_k: 64
        - k: 32
    - cohere rerank
        - k: 32
        - top_n: 8
        - model: rerank-multilingual-v3.0
    - LongContext Reorder

### **[RUN 5]**

- 성능: 테스트 1, 2, 3, 4, 6, 7, 8 정답 / 5 데이터 확인 불가

- textsplitter
    - chunk_size=512
    - chunk_overlap=50  

- retriever: 
    - type: mmr
    - kwargs
        - lambda_mult: 0.4
        - fetch_k: 96
        - k: 48
    - cohere rerank
        - top_n: 10
        - model: rerank-multilingual-v3.0
    - LongContext Reorder  

- prompt:  
    - 답변 프롬프트 추가  
        - 당신은 '모두의연구소(모두연)' 수강생들의 비서입니다.
        - 사용자 입력에 포함된 사실은 근거로 사용하지 마세요.  
        - 현재 시간은 {today} (KST)입니다. '어제, 내일' 등의 표현은 {today}를 기준으로 파악하세요.  

- 기타
    - 현재 시간 메서드 추가  

### **[RUN 6 - 예정]**

- MultiRetrievalQAChain 활용?
    - 인덱싱에 대해 description을 자세히 작성하고, LLM에게 쿼리에 맞는 적절한 옵션(파일)을 고르게끔 함  
    - ex. 

- text splitter 파라미터 실험  

- retriever 파라미터 실험  

In [14]:
# ===================
# 테스트 1
# ===================

#채팅 히스토리를 적재하기 위한 리스트
chat_history = []

question = "몇 시부터 지각이야?"

#첫 질문에 답변하기 위한 rag_chain 실행
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history, "today": today_str})

#첫 질문과 답변을 채팅 히스토리로 저장
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1["answer"])])

second_question = "조퇴는?"

#두번째 질문 입력 시에는 첫번째 질문-답변이 저장된 chat_history가 삽입됨
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history, "today": today_str})

print(ai_msg_2["answer"])

조퇴는 14시 30분부터 17시 49분 퇴실하는 경우를 말합니다.


In [15]:
# ===================
# 테스트 2
# ===================

chat_history = []

question = "몇 시부터 지각이야?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history, "today": today_str})

chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1["answer"])])

second_question = "휴가 신청서 작성해야 한다고 들었는데, 어떤 서류야?"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history, "today": today_str})

print(ai_msg_2["answer"])

휴가 신청서 작성 시 제출해야 하는 서류는 '휴가신청서(데싸 5기).docx'입니다. 또한, 당일 또는 발생일 이후 사용은 불가하며, 휴가 사용 전 신청서 제출이 필수입니다.


In [16]:
# ===================
# 테스트 3
# ===================

question = "휴가쓰고 싶어. 휴가 신청하려면 어떻게 해야해?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

휴가를 신청하려면 휴가신청서(데싸 5기)를 작성하여 제출해야 합니다. 당일 또는 발생일 이후에는 사용이 불가하니 사전에 신청하는 것이 중요합니다. 서류 제출 후에는 운영 매니저에게 확인을 받으세요.


In [18]:
# ===================
# 테스트 4
# ===================

chat_history = []

question = "데이터 사이언스 5기 동료들 중 MBTI가 ISTJ 인 사람은 누구야?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history, "today": today_str})

chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1["answer"])])

second_question = "그 사람들의 취미는 뭐야?"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history, "today": today_str})

print(ai_msg_2["answer"])

추영재의 취미는 운동(풋살, 러닝, 헬스)과 독서입니다.


In [19]:
# ===================
# 테스트 4 - BONUS
# ===================

question = "데이터 사이언스 5기 동료들 중 MBTI가 ISTJ 인 사람은 누구야?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

MBTI가 ISTJ인 사람은 추영재입니다.


In [20]:
# ===================
# 테스트 5
# ===================

question = "나는 데싸5기 손호진이야. 오늘은 2025년 10월 26일인데, 지금까지 내가 총 조퇴와 결석을 몇 번 했는지 궁금해."
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

문서에서 관련 내용이 확인되지 않습니다.


In [21]:
# ===================
# 테스트 6
# ===================

question = "모두연이 뭐야?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

모두연은 자유롭게 연구하고 배우는 커뮤니티로, 경쟁 없이 집단 지성의 힘을 활용하여 함께 학습하는 공간입니다. 이곳은 기존의 교육 방식을 바꾸고, 상생과 성장을 추구하는 사람들을 위한 장소입니다. 모두연은 스스로 질문하고 탐구하는 과정을 통해 지식을 나누고 성장하는 것을 목표로 합니다.


In [22]:
# ===================
# 테스트 7
# ===================

question = "모두의 연구소의 핵심 슬로건이 뭐야?"
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

모두의 연구소의 핵심 슬로건은 ‘쉐벨그투’로, 이는 '쉐어 벨류 그로우 투게더(SHARE VALUE, GROW TOGETHER)'의 약자입니다.


In [23]:
# ===================
# 테스트 8
# ===================

question = "쉐벨그투에 대해 알려줘."
result = rag_chain.invoke({"input": question, "chat_history": [], "today": today_str})

print(result["answer"])

쉐벨그투는 '쉐어 벨류 그로우 투게더(SHARE VALUE, GROW TOGETHER)'의 약자로, 각 영어 단어의 첫 한글 발음 모음으로 구성된 핵심 슬로건입니다. 이 슬로건은 모두연의 가치와 목표를 나타내며, 함께 가치를 나누고 성장하자는 의미를 담고 있습니다.
